In [38]:
import requests
import pandas as pd 
import json
import calendar

series_list1 = {
'LAUST010000000000003':'Alabama', 'LAUST020000000000003':'Alaska', 'LAUST040000000000003':'Arizona', 'LAUST050000000000003':'Arkansas', 'LAUST060000000000003':'California', 
'LAUST080000000000003':'Colorado', 'LAUST090000000000003':'Connecticut', 'LAUST100000000000003':'Delaware', 'LAUST120000000000003':'Florida', 'LAUST130000000000003':'Georgia', 
'LAUST150000000000003':'Hawaii', 'LAUST160000000000003':'Idaho', 'LAUST170000000000003':'Illinois', 'LAUST180000000000003':'Indiana', 
'LAUST190000000000003':'Iowa', 'LAUST200000000000003':'Kansas', 'LAUST210000000000003':'Kentucky', 'LAUST220000000000003':'Louisiana', 'LAUST230000000000003':'Maine', 
'LAUST240000000000003':'Maryland', 'LAUST250000000000003':'Massachusetts', 'LAUST260000000000003':'Michigan', 'LAUST270000000000003':'Minnesota', 'LAUST280000000000003':'Mississippi',
'LAUST290000000000003':'Missouri'} 
series_list2 = {
'LAUST300000000000003':'Montana', 'LAUST310000000000003':'Nebraska', 'LAUST320000000000003':'Nevada', 'LAUST330000000000003':'New Hampshire', 'LAUST340000000000003':'New Jersey', 
'LAUST350000000000003':'New Mexico', 'LAUST360000000000003':'New York', 'LAUST370000000000003':'North Carolina', 'LAUST380000000000003':'North Dakota', 'LAUST390000000000003':'Ohio', 'LAUST400000000000003':'Oklahoma', 
'LAUST410000000000003':'Oregon', 'LAUST420000000000003':'Pennsylvania', 'LAUST440000000000003':'Rhode Island', 'LAUST450000000000003':'South Carolina', 'LAUST460000000000003':'South Dakota', 'LAUST470000000000003':'Tennessee', 
'LAUST480000000000003':'Texas', 'LAUST490000000000003':'Utah', 'LAUST500000000000003':'Vermont', 'LAUST510000000000003':'Virginia', 
'LAUST530000000000003':'Washington', 'LAUST540000000000003':'West Virginia', 'LAUST550000000000003':'Wisconsin', 'LAUST560000000000003':'Wyoming',
}
df = pd.DataFrame(columns= ["State","Year","Month","Unemployment Rate"])

In [39]:
headers = {'Content-type': 'application/json'}
data1 = json.dumps({"seriesid": list(series_list1.keys()),"startyear":"2019", "endyear":"2021"})
p = requests.post('https://api.bls.gov/publicAPI/v1/timeseries/data/', data=data1, headers=headers)
json_data1 = json.loads(p.text)
data2 = json.dumps({"seriesid": list(series_list2.keys()),"startyear":"2019", "endyear":"2021"})
p = requests.post('https://api.bls.gov/publicAPI/v1/timeseries/data/', data=data2, headers=headers)
json_data2 = json.loads(p.text)

for series in json_data1['Results']['series']:
    state = series_list1[series['seriesID']]
    for item in series['data']:
        year = item['year']
        month = calendar.month_name[int(item['period'][1:3])]
        value = item['value']
        df = df.append({'State':state,'Year':year,'Month':month,'Unemployment Rate':value},ignore_index=True)

for series in json_data2['Results']['series']:
    state = series_list2[series['seriesID']]
    for item in series['data']:
        year = item['year']
        month = calendar.month_name[int(item['period'][1:3])]
        value = item['value']
        df = df.append({'State':state,'Year':year,'Month':month,'Unemployment Rate':value},ignore_index=True)
df

,State,Year,Month,Unemployment Rate
0,Alabama,2021,December,2.6
1,Alabama,2021,November,2.6
2,Alabama,2021,October,2.9
3,Alabama,2021,September,3.0
4,Alabama,2021,August,3.7
...,...,...,...,...
1795,Wyoming,2019,May,3.3
1796,Wyoming,2019,April,3.4
1797,Wyoming,2019,March,4.0
1798,Wyoming,2019,February,3.9


In [48]:
df_2019 = df.loc[df['Year']== '2019'].copy()
df_2019['Year Month'] = df_2019['Year'] +' '+ df_2019['Month']
df_2019 = df_2019[['Year Month', 'State','Unemployment Rate']]
df_2019.set_index('Year Month')

,State,Unemployment Rate
Year Month,,
2019 December,Alabama,2.8
2019 November,Alabama,2.7
2019 October,Alabama,2.7
2019 September,Alabama,2.8
2019 August,Alabama,3.2
...,...,...
2019 May,Wyoming,3.3
2019 April,Wyoming,3.4
2019 March,Wyoming,4.0


In [47]:
df_2020 = df.loc[df['Year']== '2020'].copy()
df_2020['Year Month'] = df_2020['Year'] +' '+ df_2020['Month']
df_2020 = df_2020[['Year Month', 'State','Unemployment Rate']]
df_2020.set_index('Year Month')

,State,Unemployment Rate
Year Month,,
2020 December,Alabama,4.1
2020 November,Alabama,4.4
2020 October,Alabama,4.9
2020 September,Alabama,6.0
2020 August,Alabama,6.7
...,...,...
2020 May,Wyoming,8.8
2020 April,Wyoming,5.3
2020 March,Wyoming,6.6


In [49]:
df_2021 = df.loc[df['Year']== '2021'].copy()
df_2021['Year Month'] = df_2021['Year'] +' '+ df_2021['Month']
df_2021 = df_2021[['Year Month', 'State','Unemployment Rate']]
df_2021.set_index('Year Month')

,State,Unemployment Rate
Year Month,,
2021 December,Alabama,2.6
2021 November,Alabama,2.6
2021 October,Alabama,2.9
2021 September,Alabama,3.0
2021 August,Alabama,3.7
...,...,...
2021 May,Wyoming,4.7
2021 April,Wyoming,5.0
2021 March,Wyoming,5.4
